In [367]:
%load_ext autoreload
%autoreload 2
import bert
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import spike_queries
from termcolor import colored

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [326]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [393]:
def load_results(fname):
    
    with open(fname, "r", encoding = "utf-8") as f:
        sents = f.readlines()
    
    sents = [s.strip().split("\t")[-1] for s in sents]
    original, results = sents[0], sents[1:]
    return original, results


def get_spike_results_arguments_representations(model, spike_results, layers):
    
    sents = spike_results["sentence_text"].tolist()
    arg1_idx_start = spike_results["arg1_first_index"].to_numpy().astype(int)
    arg2_idx_start = spike_results["arg2_first_index"].to_numpy().astype(int)
    arg1_idx_end = spike_results["arg1_last_index"].to_numpy().astype(int)
    arg2_idx_end = spike_results["arg2_last_index"].to_numpy().astype(int)
    
    arg1_rep = []
    arg2_rep = []
    
    for s, arg1_start, arg2_start, arg1_end, arg2_end in zip(sents, arg1_idx_start, arg2_idx_start, arg1_idx_end, arg2_idx_end):
        idx_to_mask = [arg1_start, arg2_start, arg1_end, arg2_end]
        H, _, _, orig2tok = model.encode(s, layers = layers)
        h1, h2 = H[orig2tok[arg1_start]:orig2tok[arg1_end] + 1], H[orig2tok[arg2_start]:orig2tok[arg2_end] + 1]
        
        h1 = np.mean(h1, axis = 0)
        h2 = np.mean(h2, axis = 0)
        
        arg1_rep.append(h1)
        arg2_rep.append(h2)
        
    arg1_mean = np.mean(arg1_rep, axis = 0)
    arg2_mean = np.mean(arg2_rep, axis = 0)
    
    return arg1_mean, arg2_mean
    

def print_sentence_nicely(sentence: str, ind1, ind2):
    
    arg1_sign = "**"
    arg2_sign = "++"
    
    if not ind2 > ind1: 
        nd1, ind2 = ind2, ind1
        arg1_sign, arg2_sign = arg2_sign, arg1_sign
        
    splitted = sentence.split(" ")
    before_arg1 = " ".join(splitted[:ind1])
    arg1 = splitted[ind1]
    arg2 = splitted[ind2]
    between = " ".join(splitted[ind1 + 1: ind2])
    suffix = " ".join(splitted[ind2+1:])
    
    return before_arg1 +  " " + arg1_sign + colored(arg1, "red") + arg1_sign + " " + between + " " + arg2_sign + colored(arg2, "blue") + arg2_sign + " " + suffix



def main(filename, layers = [-1]):
    

    query, results1 = load_results(filename)
    spike_results = spike_queries.perform_query(query, dataset_name = "covid19", num_results = 50, query_type = "syntactic")
    spike_results = spike_results[spike_results['sentence_text'].notnull()]
    arg1_rep, arg2_rep = get_spike_results_arguments_representations(model, spike_results, layers)
    
    print(color.BOLD + "QUERY" + color.END + ":\n{}".format(query))
    
    first, first_ind1, first_ind2 = spike_results["sentence_text"].tolist()[-1], int(spike_results["arg1_first_index"].tolist()[-1]), int(spike_results["arg2_first_index"].tolist()[-1])
    print(color.BOLD + "\nFIRST SPIKE RESULT" + color.END + ":\n{}".format(print_sentence_nicely(first, first_ind1, first_ind2)))
    print(color.BOLD + "\nAUGMENTATION RESULTS:\n" + color.END)
    
    for s in results1:
        H, tokenized_text, tok_to_orig_map, orig2tok = model.encode(s, layers = layers)
        #print("H shape: {}".format(H.shape))
        sims_arg1 = cosine_similarity([arg1_rep], H[1:-2])[0]
        sims_arg2 = cosine_similarity([arg2_rep], H[1:-2])[0]
        arg1_ind = np.argmax(sims_arg1) + 1
        
        sims_arg2[np.argmax(sims_arg1)] = -1
        arg2_ind = np.argmax(sims_arg2) + 1
        print(arg1_ind, arg2_ind)
        if arg1_ind not in tok_to_orig_map or arg2_ind not in tok_to_orig_map:
            print("ERROR")
            continue
        print(print_sentence_nicely(s, tok_to_orig_map[arg1_ind], tok_to_orig_map[arg2_ind]))
        print("-----------------------------------")
    
    print(color.BOLD + "=======================================================" + color.END)


In [348]:
model = bert.BertEncoder("cpu", "scibert")

loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/allenai/scibert_scivocab_uncased/config.json from cache at /home/shauli/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d813f7395e7ea533039e02deb1723d8fd9d8ba655391a01a69ad6223d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

Model name 'allenai/scibert_scivocab_uncased' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-

In [387]:
main("results1.txt", layers = [-1])

QUERY:
<>arg1:virus $infection $causes a <>arg2:condition .

FIRST SPIKE RESULT:
178 Thus , **leishmania** infection causes ++secondary++ infections , and activation/reactivation of dormant infections that can lead to severe or lethal outcomes through multiple mechanisms .

AUGMENTATION RESULTS:

5 15
In humans , **JEV** infection can cause Japanese encephalitis ( JE ) with ++severe++ central nervous system disorders .
-----------------------------------
3 18
Hepatotropic **virus** , like MHV-3 infection in mice , can induce exaggerated ++inflammation++ in the liver and cause life-threatening viral FH .
-----------------------------------
11 15
Human metapneumovirus ( hMPV ) **infection** causes respiratory tract ++disease++ similar to that observed during human respiratory syncytial virus infection ( hRSV ) .
-----------------------------------
6 13
In humans , CHIKV **infections** cause a debilitating ++disease++ with acute febrile illness and long-term polyarthralgia .
-------------

Accuracy: 16/20

In [388]:
main("results2.txt", [-1])

QUERY:
infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

FIRST SPIKE RESULT:
12 , 27 , [ 33 ] [ 34 ] [ 35 ] **Puppies** that are nursing from CHV-seronegative dams usually develop the fatal ++multisystemic++ disease , while puppies that suckle from CHV-seropositive dams remain asymptomatic but still become infected .

AUGMENTATION RESULTS:

19 24
After an incubation period of 5 to 14 days , SARS-CoV-2-infected **people** commonly manifest features of ++pneumonia++ , including fever , dry cough , dyspnoea , myalgia and fatigue .
-----------------------------------
11 23
After an incubation period of 2 to 3 days , **patients** who have pneumonic plague typically develop fulminant ++pneumonia++ , with malaise , high fever , cough , hemoptysis , and septicemia with ecchymoses and extremity necrosis .
-----------------------------------
1 12
 **Patients** suffering from severe DENV infection often exhibit encephalopathy and ++encephalitis++ .
--------------------

Accuracy: 18/20

In [395]:
main("results3.txt", [-1])

QUERY:
a arg1:subset of patients  $progress to arg2:hemorrhagic fever

FIRST SPIKE RESULT:
The **disease** may progress across the bone of the skull base to the ++jugular++ foramen and hypoglossal canal , resulting in palsies of the ninth , tenth , eleventh and twelfth cranial nerves .

AUGMENTATION RESULTS:

2 1
Severe ++cases++  **cases** may progress to a capillary leak syndrome with septic shock , rash , facial and neck swelling , and multi-organ system failure .
-----------------------------------
6 10
When symptoms occur , IPS **may** rapidly progress to ++pulmonary++ dysfunction requiring mechanical ventilation .
-----------------------------------
13 1
Clinical signs are nonspecific in the early stages , but as the ++disease++  **disease** progresses to involve more of the liver and impair regeneration , icterus , ascites , and hepatic encephalopathy may develop as typical correlates with hepatic insufficiency .
-----------------------------------
2 25
The **patients** can deve

Accuarcy: 13/20

In [390]:
main("results4.txt", [-1])

QUERY:
people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

FIRST SPIKE RESULT:
However , children with inborn **errors** in TLR3 signaling ( 54 ) ( 55 ) ( 56 ) or defects resulting in abnormal signaling through several antiviral TLRs including 3 , 7 , 8 , 9 ( e.g. UNC-93B deficiency)(57 ) are susceptible to ++HSVE++ .

AUGMENTATION RESULTS:

4 9
Patients with lung **cancer** are highly susceptible to ++infection++ compared to healthy individuals because of systemic immunosuppression induced by malignancy and anticancer therapy .
-----------------------------------
4 9
Patients with lung **cancer** are highly susceptible to ++infection++ compared to healthy individuals because of systemic immunosuppression induced by malignancy and anticancer therapy .
-----------------------------------
1 8
 **Patients** with ARDS triggered by viral ++infection++ , in particular influenza , are prone to invasive aspergillosis even in absence of prior immunodeficiency .
----------------------

Accuracy: 17/20

In [391]:
main("results5.txt", [-1])

QUERY:
arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

FIRST SPIKE RESULT:
There are cases of ADEM or even fulminant presentation such as AHL where **steroids** alone are not sufficient for suppressing ++inflammation++ and improving clinical findings .

AUGMENTATION RESULTS:

19 31
However , while tocilizumab is a promising agent against COVID-19 , **it** is not an appropriate agent in patients with active or latent ++tuberculosis++ , bacterial and fungal infections , multi-organ failure , and gastrointestinal perforation [ 7 ] .
-----------------------------------
18 25
There are cases of ADEM or even fulminant presentation such as AHL where **steroids** alone are not sufficient for suppressing ++inflammation++ and improving clinical findings .
-----------------------------------
5 13
Thus , **ribavirin** may not be useful for treating ++SARS++ infections because of its questionable efficacy and because of its known toxicity ( reviewed by van Vonderen et al. , 2003 ; 

Accuracy: 13/20

In [394]:
main("results6.txt", [-1])

QUERY:
arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

FIRST SPIKE RESULT:
In conclusion , hepatic **injury** is likely a complication of COVID-19 ++infection++ .

AUGMENTATION RESULTS:

3 9
Acute kidney **injury** is a possible manifestation in ++severe++ forms of COVID-19 , conferring a high mortality 7 .
-----------------------------------
10 12
Coagulation dysfunction is one of the major causes for **death** in ++patients++ with severe COVID-19 [ 4 ] .
-----------------------------------
1 12
ERROR
1 8
 **Hypoxia** is the most common manifestation of ++patients++ with COVID-19 , especially in severely or critically ill patients .
-----------------------------------
4 17
Early-onset **pneumonia** is a common and severe complication that is related to aspiration in ++patients++ with GBS .
-----------------------------------
3 9
 **Barotrauma** is a common complication in ++patients++ with SARS .
-----------------------------------
5 6
COVID-19 **infection**  ++is++ 

Accuracy: 17/20